#### ML on Augmented DataSet

##### XGboost

In [ ]:
import os, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    classification_report, confusion_matrix, roc_curve, auc,
    precision_recall_curve, average_precision_score
)
from xgboost import XGBClassifier
import joblib


CSV_PATH   = "/kaggle/working/features_256d_efficientnet.csv"
OUT_DIR    = "/kaggle/working/Customized CNN/XGB"
os.makedirs(OUT_DIR, exist_ok=True)
REPORT_OUT = os.path.join(OUT_DIR, "xgb_report.json")
MODEL_VAL_BEST = os.path.join(OUT_DIR, "xgb_valbest.joblib")
MODEL_FINAL    = os.path.join(OUT_DIR, "xgb_final_trainval.joblib")
SEARCH_CSV     = os.path.join(OUT_DIR, "xgb_val_search_results.csv")


df = pd.read_csv(CSV_PATH)
feat_cols = [c for c in df.columns if c.startswith("f") and c[1:].isdigit()]
X = df[feat_cols].values.astype(np.float32)
y = df["class_idx"].values.astype(np.int64)

class_map = df.sort_values("class_idx")[["class_idx","label"]].drop_duplicates()
class_names = class_map.set_index("class_idx")["label"].reindex(sorted(class_map["class_idx"])).tolist()
n_classes = len(np.unique(y))
rng = 42


X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=rng
)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.125, stratify=y_trainval, random_state=rng
)

scaler = StandardScaler().fit(X_train)
X_train_s = scaler.transform(X_train)
X_val_s   = scaler.transform(X_val)
X_test_s  = scaler.transform(X_test)

def make_xgb(params):
    return XGBClassifier(
        objective="multi:softprob",
        num_class=n_classes,
        eval_metric="mlogloss",
        tree_method="hist",
        random_state=rng,
        n_jobs=-1,
        **params
    )

PARAMS = {
    "learning_rate":     [0.03, 0.06],     
    "max_depth":         [3, 4, 5],        
    "min_child_weight":  [1, 3],           
    "subsample":         [0.8, 1.0],
    "colsample_bytree":  [0.8, 1.0],
    "reg_alpha":         [0.0, 1e-3],     
    "reg_lambda":        [1.0],           
}

search_rows, best = [], {"acc": -1, "params": None, "model": None}

BASE_N_ESTIMATORS = 2000
EARLY_STOP_ROUNDS = 50

keys = list(PARAMS.keys())
grids = [dict(zip(keys, values)) for values in product(*[PARAMS[k] for k in keys])]

for p in grids:
    params = dict(
        n_estimators=BASE_N_ESTIMATORS,
        **p
    )
    model = make_xgb(params)
    model.fit(
        X_train_s, y_train,
        eval_set=[(X_val_s, y_val)],
        verbose=False,
        early_stopping_rounds=EARLY_STOP_ROUNDS
    )
    yv_pred = model.predict(X_val_s)
    acc = accuracy_score(y_val, yv_pred)
    f1m = f1_score(y_val, yv_pred, average="macro")

    search_rows.append({**p, "best_iteration": int(getattr(model, "best_iteration", model.n_estimators-1)),
                        "val_accuracy": acc, "val_f1_macro": f1m})

    if acc > best["acc"]:
        best.update({"acc": acc, "params": params, "model": model})


pd.DataFrame(search_rows).sort_values(["val_accuracy","val_f1_macro"], ascending=False).to_csv(SEARCH_CSV, index=False)
joblib.dump({"scaler": scaler, "model": best["model"]}, MODEL_VAL_BEST)


def save_confmat_and_reports(Xs, y_true, model, prefix):
    y_proba = model.predict_proba(Xs)
    y_pred  = y_proba.argmax(1)

    acc   = accuracy_score(y_true, y_pred)
    f1m   = f1_score(y_true, y_pred, average="macro")
    f1w   = f1_score(y_true, y_pred, average="weighted")
    precm = precision_score(y_true, y_pred, average="macro")
    precw = precision_score(y_true, y_pred, average="weighted")
    recm  = recall_score(y_true, y_pred, average="macro")
    recw  = recall_score(y_true, y_pred, average="weighted")

    pd.DataFrame(classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
                 ).transpose().to_csv(os.path.join(OUT_DIR, f"{prefix}_classification_report.csv"))

    cm = confusion_matrix(y_true, y_pred)
    pd.DataFrame(cm, index=class_names, columns=class_names).to_csv(os.path.join(OUT_DIR, f"{prefix}_confusion_matrix.csv"))
    plt.figure(figsize=(6,5)); plt.imshow(cm, interpolation='nearest'); plt.title(f"Confusion Matrix — {prefix}")
    plt.colorbar(); ticks=np.arange(len(class_names))
    plt.xticks(ticks, class_names, rotation=45, ha="right"); plt.yticks(ticks, class_names)
    th=cm.max()/2
    for i in range(cm.shape[0]):
      for j in range(cm.shape[1]):
        plt.text(j,i,str(cm[i,j]),ha="center",va="center",color="white" if cm[i,j]>th else "black")
    plt.tight_layout(); plt.ylabel("True"); plt.xlabel("Pred")
    plt.savefig(os.path.join(OUT_DIR, f"{prefix}_confusion_matrix.png"), dpi=160, bbox_inches="tight"); plt.close()

    metrics_extra = {}
    if n_classes > 1:
        y_bin = label_binarize(y_true, classes=np.arange(n_classes))
        fpr, tpr, roc_auc, prec, rec, ap = {}, {}, {}, {}, {}, {}
        for c in range(n_classes):
            fpr[c], tpr[c], _ = roc_curve(y_bin[:, c], y_proba[:, c]); roc_auc[c] = auc(fpr[c], tpr[c])
            prec[c], rec[c], _ = precision_recall_curve(y_bin[:, c], y_proba[:, c]); ap[c] = average_precision_score(y_bin[:, c], y_proba[:, c])
        fpr["micro"], tpr["micro"], _ = roc_curve(y_bin.ravel(), y_proba.ravel()); roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
        prec["micro"], rec["micro"], _ = precision_recall_curve(y_bin.ravel(), y_proba.ravel()); ap["micro"] = average_precision_score(y_bin, y_proba, average="micro")
        all_fpr = np.unique(np.concatenate([fpr[c] for c in range(n_classes)])); mean_tpr = np.zeros_like(all_fpr)
        for c in range(n_classes): mean_tpr += np.interp(all_fpr, fpr[c], tpr[c])
        mean_tpr /= n_classes; roc_auc["macro"] = auc(all_fpr, mean_tpr); ap["macro"] = np.mean([ap[c] for c in range(n_classes)])
        rows = []; 
        for key in list(range(n_classes)) + ["micro"]:
            for xi, yi in zip(fpr[key], tpr[key]): rows.append({"curve": f"ROC_{key}", "fpr": float(xi), "tpr": float(yi)})
        for xi, yi in zip(all_fpr, mean_tpr): rows.append({"curve": "ROC_macro", "fpr": float(xi), "tpr": float(yi)})
        pd.DataFrame(rows).to_csv(os.path.join(OUT_DIR, f"{prefix}_roc_points.csv"), index=False)
        rows = []
        for key in list(range(n_classes)) + ["micro"]:
            for pi, ri in zip(prec[key], rec[key]): rows.append({"curve": f"PR_{key}", "precision": float(pi), "recall": float(ri)})
        pd.DataFrame(rows).to_csv(os.path.join(OUT_DIR, f"{prefix}_pr_points.csv"), index=False)

        plt.figure(figsize=(7,6))
        for c in range(n_classes): plt.plot(fpr[c], tpr[c], lw=1.2, label=f"{class_names[c]} (AUC={roc_auc[c]:.3f})")
        plt.plot(fpr["micro"], tpr["micro"], lw=2, linestyle="--", label=f"micro (AUC={roc_auc['micro']:.3f})")
        plt.plot([0,1],[0,1],"k--", lw=1); plt.xlim([0,1]); plt.ylim([0,1.05])
        plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(f"ROC — {prefix}")
        plt.legend(loc="lower right", fontsize=8); plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, f"{prefix}_roc_curves.png"), dpi=160); plt.close()

        plt.figure(figsize=(7,6))
        for c in range(n_classes): plt.plot(rec[c], prec[c], lw=1.2, label=f"{class_names[c]} (AP={ap[c]:.3f})")
        plt.plot(rec["micro"], prec["micro"], lw=2, linestyle="--", label=f"micro (AP={ap['micro']:.3f})")
        plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(f"PR — {prefix}")
        plt.legend(loc="lower left", fontsize=8); plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, f"{prefix}_pr_curves.png"), dpi=160); plt.close()

        metrics_extra = {
            "roc_auc_per_class": {class_names[c]: float(roc_auc[c]) for c in range(n_classes)},
            "roc_auc_micro": float(roc_auc["micro"]),
            "roc_auc_macro": float(roc_auc["macro"]),
            "ap_per_class": {class_names[c]: float(ap[c]) for c in range(n_classes)},
            "ap_micro": float(ap["micro"]),
            "ap_macro": float(ap["macro"]),
        }

    return {
        "accuracy": float(acc),
        "f1_macro": float(f1m),
        "f1_weighted": float(f1w),
        "precision_macro": float(precm),
        "precision_weighted": float(precw),
        "recall_macro": float(recm),
        "recall_weighted": float(recw),
        **metrics_extra
    }

val_summary = save_confmat_and_reports(X_val_s, y_val, best["model"], prefix="val_xgb")
joblib.dump({"scaler": scaler, "model": best["model"]}, MODEL_VAL_BEST)

p = best["params"]
final_model = make_xgb(p)
final_model.fit(np.vstack([X_train_s, X_val_s]), np.concatenate([y_train, y_val]), verbose=False)
joblib.dump({"scaler": scaler, "model": final_model}, MODEL_FINAL)

test_summary = save_confmat_and_reports(X_test_s, y_test, final_model, prefix="test_xgb")

report = {
    "split_sizes": {"train": int(X_train.shape[0]), "val": int(X_val.shape[0]), "test": int(X_test.shape[0])},
    "val_search_best_params": p,
    "val_metrics": val_summary,
    "test_metrics": test_summary,
    "classes": class_names
}
with open(REPORT_OUT, "w") as f:
    json.dump(report, f, indent=2)

print("\nSaved artifacts to:", OUT_DIR)
print("Best VAL acc:", f"{val_summary['accuracy']:.4f}")
print("Final TEST acc:", f"{test_summary['accuracy']:.4f}")
print("Report:", REPORT_OUT)


##### SVG

In [ ]:

import os, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    classification_report, confusion_matrix, roc_curve, auc,
    precision_recall_curve, average_precision_score
)
import joblib

CSV_PATH   = "/kaggle/working/features_256d_efficientnet.csv"
OUT_DIR    = "/kaggle/working/Customized CNN/SVM"
os.makedirs(OUT_DIR, exist_ok=True)
REPORT_OUT = os.path.join(OUT_DIR, "svm_report.json")
MODEL_VAL_BEST = os.path.join(OUT_DIR, "svm_valbest.joblib")
MODEL_FINAL    = os.path.join(OUT_DIR, "svm_final_trainval.joblib")
SEARCH_CSV     = os.path.join(OUT_DIR, "svm_val_search_results.csv")

df = pd.read_csv(CSV_PATH)
feat_cols = [c for c in df.columns if c.startswith("f") and c[1:].isdigit()]
X = df[feat_cols].values.astype(np.float32)
y = df["class_idx"].values.astype(np.int64)

class_map = df.sort_values("class_idx")[["class_idx","label"]].drop_duplicates()
class_names = class_map.set_index("class_idx")["label"].reindex(sorted(class_map["class_idx"])).tolist()
n_classes = len(np.unique(y))
rng = 42

X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=rng
)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.125, stratify=y_trainval, random_state=rng
)

# Standardize
scaler = StandardScaler().fit(X_train)
X_train_s = scaler.transform(X_train)
X_val_s   = scaler.transform(X_val)
X_test_s  = scaler.transform(X_test)

def make_svm(C, gamma):
    return SVC(kernel="rbf", C=C, gamma=gamma, probability=True, random_state=rng)

C_LIST     = np.logspace(-2, 3, 12)    
GAMMA_LIST = np.logspace(-5, 1, 13)  

rows, best = [], {"acc": -1, "params": None, "model": None}
for C in C_LIST:
    for g in GAMMA_LIST:
        model = make_svm(C, g)
        model.fit(X_train_s, y_train)
        yv_pred = model.predict(X_val_s)
        acc = accuracy_score(y_val, yv_pred)
        f1m = f1_score(y_val, yv_pred, average="macro")
        rows.append({"C": C, "gamma": g, "val_accuracy": acc, "val_f1_macro": f1m})
        if acc > best["acc"]:
            best.update({"acc": acc, "params": {"C": float(C), "gamma": float(g)}, "model": model})

pd.DataFrame(rows).sort_values(["val_accuracy","val_f1_macro"], ascending=False).to_csv(SEARCH_CSV, index=False)
joblib.dump({"scaler": scaler, "model": best["model"]}, MODEL_VAL_BEST)

def save_confmat_and_reports(Xs, y_true, model, prefix):
    y_proba = model.predict_proba(Xs)
    y_pred  = y_proba.argmax(1)

    acc   = accuracy_score(y_true, y_pred)
    f1m   = f1_score(y_true, y_pred, average="macro")
    f1w   = f1_score(y_true, y_pred, average="weighted")
    precm = precision_score(y_true, y_pred, average="macro")
    precw = precision_score(y_true, y_pred, average="weighted")
    recm  = recall_score(y_true, y_pred, average="macro")
    recw  = recall_score(y_true, y_pred, average="weighted")

    pd.DataFrame(classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
                 ).transpose().to_csv(os.path.join(OUT_DIR, f"{prefix}_classification_report.csv"))

    cm = confusion_matrix(y_true, y_pred)
    pd.DataFrame(cm, index=class_names, columns=class_names).to_csv(os.path.join(OUT_DIR, f"{prefix}_confusion_matrix.csv"))
    plt.figure(figsize=(6,5)); plt.imshow(cm, interpolation='nearest'); plt.title(f"Confusion Matrix — {prefix}")
    plt.colorbar(); ticks=np.arange(len(class_names))
    plt.xticks(ticks, class_names, rotation=45, ha="right"); plt.yticks(ticks, class_names)
    th=cm.max()/2
    for i in range(cm.shape[0]):
      for j in range(cm.shape[1]):
        plt.text(j,i,str(cm[i,j]),ha="center",va="center",color="white" if cm[i,j]>th else "black")
    plt.tight_layout(); plt.ylabel("True"); plt.xlabel("Pred")
    plt.savefig(os.path.join(OUT_DIR, f"{prefix}_confusion_matrix.png"), dpi=160, bbox_inches="tight"); plt.close()

    metrics_extra = {}
    if n_classes > 1:
        y_bin = label_binarize(y_true, classes=np.arange(n_classes))
        fpr, tpr, roc_auc, prec, rec, ap = {}, {}, {}, {}, {}, {}
        for c in range(n_classes):
            fpr[c], tpr[c], _ = roc_curve(y_bin[:, c], y_proba[:, c]); roc_auc[c] = auc(fpr[c], tpr[c])
            prec[c], rec[c], _ = precision_recall_curve(y_bin[:, c], y_proba[:, c]); ap[c] = average_precision_score(y_bin[:, c], y_proba[:, c])
        fpr["micro"], tpr["micro"], _ = roc_curve(y_bin.ravel(), y_proba.ravel()); roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
        prec["micro"], rec["micro"], _ = precision_recall_curve(y_bin.ravel(), y_proba.ravel()); ap["micro"] = average_precision_score(y_bin, y_proba, average="micro")
        all_fpr = np.unique(np.concatenate([fpr[c] for c in range(n_classes)])); mean_tpr = np.zeros_like(all_fpr)
        for c in range(n_classes): mean_tpr += np.interp(all_fpr, fpr[c], tpr[c])
        mean_tpr /= n_classes; roc_auc["macro"] = auc(all_fpr, mean_tpr); ap["macro"] = np.mean([ap[c] for c in range(n_classes)])
        rows = []; 
        for key in list(range(n_classes)) + ["micro"]:
            for xi, yi in zip(fpr[key], tpr[key]): rows.append({"curve": f"ROC_{key}", "fpr": float(xi), "tpr": float(yi)})
        for xi, yi in zip(all_fpr, mean_tpr): rows.append({"curve": "ROC_macro", "fpr": float(xi), "tpr": float(yi)})
        pd.DataFrame(rows).to_csv(os.path.join(OUT_DIR, f"{prefix}_roc_points.csv"), index=False)
        rows = []
        for key in list(range(n_classes)) + ["micro"]:
            for pi, ri in zip(prec[key], rec[key]): rows.append({"curve": f"PR_{key}", "precision": float(pi), "recall": float(ri)})
        pd.DataFrame(rows).to_csv(os.path.join(OUT_DIR, f"{prefix}_pr_points.csv"), index=False)

        plt.figure(figsize=(7,6))
        for c in range(n_classes): plt.plot(fpr[c], tpr[c], lw=1.2, label=f"{class_names[c]} (AUC={roc_auc[c]:.3f})")
        plt.plot(fpr["micro"], tpr["micro"], lw=2, linestyle="--", label=f"micro (AUC={roc_auc['micro']:.3f})")
        plt.plot([0,1],[0,1],"k--", lw=1); plt.xlim([0,1]); plt.ylim([0,1.05])
        plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(f"ROC — {prefix}")
        plt.legend(loc="lower right", fontsize=8); plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, f"{prefix}_roc_curves.png"), dpi=160); plt.close()

        plt.figure(figsize=(7,6))
        for c in range(n_classes): plt.plot(rec[c], prec[c], lw=1.2, label=f"{class_names[c]} (AP={ap[c]:.3f})")
        plt.plot(rec["micro"], prec["micro"], lw=2, linestyle="--", label=f"micro (AP={ap['micro']:.3f})")
        plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(f"PR — {prefix}")
        plt.legend(loc="lower left", fontsize=8); plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, f"{prefix}_pr_curves.png"), dpi=160); plt.close()

        metrics_extra = {
            "roc_auc_per_class": {class_names[c]: float(roc_auc[c]) for c in range(n_classes)},
            "roc_auc_micro": float(roc_auc["micro"]),
            "roc_auc_macro": float(roc_auc["macro"]),
            "ap_per_class": {class_names[c]: float(ap[c]) for c in range(n_classes)},
            "ap_micro": float(ap["micro"]),
            "ap_macro": float(ap["macro"]),
        }

    return {
        "accuracy": float(acc),
        "f1_macro": float(f1m),
        "f1_weighted": float(f1w),
        "precision_macro": float(precm),
        "precision_weighted": float(precw),
        "recall_macro": float(recm),
        "recall_weighted": float(recw),
        **metrics_extra
    }

val_summary = save_confmat_and_reports(X_val_s, y_val, best["model"], prefix="val_svm")
joblib.dump({"scaler": scaler, "model": best["model"]}, MODEL_VAL_BEST)

p = best["params"]
final_model = make_svm(p["C"], p["gamma"])
final_model.fit(np.vstack([X_train_s, X_val_s]), np.concatenate([y_train, y_val]))
joblib.dump({"scaler": scaler, "model": final_model}, MODEL_FINAL)

test_summary = save_confmat_and_reports(X_test_s, y_test, final_model, prefix="test_svm")

report = {
    "split_sizes": {"train": int(X_train.shape[0]), "val": int(X_val.shape[0]), "test": int(X_test.shape[0])},
    "val_search_best_params": p,
    "val_metrics": val_summary,
    "test_metrics": test_summary,
    "classes": class_names
}
with open(REPORT_OUT, "w") as f:
    json.dump(report, f, indent=2)

print("\nSaved artifacts to:", OUT_DIR)
print("Best VAL acc:", f"{val_summary['accuracy']:.4f}")
print("Final TEST acc:", f"{test_summary['accuracy']:.4f}")
print("Report:", REPORT_OUT)


##### KNN

In [ ]:
import os, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    classification_report, confusion_matrix, roc_curve, auc,
    precision_recall_curve, average_precision_score
)
import joblib

CSV_PATH   = "/kaggle/working/features_256d_efficientnet.csv"
OUT_DIR    = "/kaggle/working/Customized CNN/KNN"
os.makedirs(OUT_DIR, exist_ok=True)
REPORT_OUT = os.path.join(OUT_DIR, "knn_report.json")
MODEL_VAL_BEST = os.path.join(OUT_DIR, "knn_valbest.joblib")
MODEL_FINAL    = os.path.join(OUT_DIR, "knn_final_trainval.joblib")
SEARCH_CSV     = os.path.join(OUT_DIR, "knn_val_search_results.csv")

df = pd.read_csv(CSV_PATH)
feat_cols = [c for c in df.columns if c.startswith("f") and c[1:].isdigit()]
X = df[feat_cols].values.astype(np.float32)
y = df["class_idx"].values.astype(np.int64)

class_map = df.sort_values("class_idx")[["class_idx","label"]].drop_duplicates()
class_names = class_map.set_index("class_idx")["label"].reindex(sorted(class_map["class_idx"])).tolist()
n_classes = len(np.unique(y))
rng = 42

X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=rng
)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.125, stratify=y_trainval, random_state=rng
)


scaler = StandardScaler().fit(X_train)
X_train_s = scaler.transform(X_train)
X_val_s   = scaler.transform(X_val)
X_test_s  = scaler.transform(X_test)


def make_knn(n_neighbors, metric, weights):
    return KNeighborsClassifier(
        n_neighbors=n_neighbors, metric=metric, weights=weights, n_jobs=-1
    )

N_NEIGHBORS = [1, 3, 5, 7, 9, 11]
METRICS     = ["euclidean", "manhattan", "cosine"]
WEIGHTS     = ["uniform", "distance"]

rows, best = [], {"acc": -1, "params": None, "model": None}
for k in N_NEIGHBORS:
    for m in METRICS:
        for w in WEIGHTS:
            model = make_knn(k, m, w)
            model.fit(X_train_s, y_train)
            yv_pred = model.predict(X_val_s)
            acc = accuracy_score(y_val, yv_pred)
            f1m = f1_score(y_val, yv_pred, average="macro")
            rows.append({"n_neighbors": k, "metric": m, "weights": w,
                         "val_accuracy": acc, "val_f1_macro": f1m})
            if acc > best["acc"]:
                best.update({"acc": acc, "params": {"n_neighbors": k, "metric": m, "weights": w}, "model": model})

pd.DataFrame(rows).sort_values(["val_accuracy","val_f1_macro"], ascending=False).to_csv(SEARCH_CSV, index=False)
joblib.dump({"scaler": scaler, "model": best["model"]}, MODEL_VAL_BEST)

def save_confmat_and_reports(Xs, y_true, model, prefix):
    y_proba = model.predict_proba(Xs)
    y_pred  = y_proba.argmax(1)

    acc   = accuracy_score(y_true, y_pred)
    f1m   = f1_score(y_true, y_pred, average="macro")
    f1w   = f1_score(y_true, y_pred, average="weighted")
    precm = precision_score(y_true, y_pred, average="macro")
    precw = precision_score(y_true, y_pred, average="weighted")
    recm  = recall_score(y_true, y_pred, average="macro")
    recw  = recall_score(y_true, y_pred, average="weighted")

    pd.DataFrame(classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
                 ).transpose().to_csv(os.path.join(OUT_DIR, f"{prefix}_classification_report.csv"))

    cm = confusion_matrix(y_true, y_pred)
    pd.DataFrame(cm, index=class_names, columns=class_names).to_csv(os.path.join(OUT_DIR, f"{prefix}_confusion_matrix.csv"))
    plt.figure(figsize=(6,5)); plt.imshow(cm, interpolation='nearest'); plt.title(f"Confusion Matrix — {prefix}")
    plt.colorbar(); ticks=np.arange(len(class_names))
    plt.xticks(ticks, class_names, rotation=45, ha="right"); plt.yticks(ticks, class_names)
    th=cm.max()/2
    for i in range(cm.shape[0]):
      for j in range(cm.shape[1]):
        plt.text(j,i,str(cm[i,j]),ha="center",va="center",color="white" if cm[i,j]>th else "black")
    plt.tight_layout(); plt.ylabel("True"); plt.xlabel("Pred")
    plt.savefig(os.path.join(OUT_DIR, f"{prefix}_confusion_matrix.png"), dpi=160, bbox_inches="tight"); plt.close()

    metrics_extra = {}
    if n_classes > 1:
        y_bin = label_binarize(y_true, classes=np.arange(n_classes))
        fpr, tpr, roc_auc, prec, rec, ap = {}, {}, {}, {}, {}, {}
        for c in range(n_classes):
            fpr[c], tpr[c], _ = roc_curve(y_bin[:, c], y_proba[:, c]); roc_auc[c] = auc(fpr[c], tpr[c])
            prec[c], rec[c], _ = precision_recall_curve(y_bin[:, c], y_proba[:, c]); ap[c] = average_precision_score(y_bin[:, c], y_proba[:, c])
        fpr["micro"], tpr["micro"], _ = roc_curve(y_bin.ravel(), y_proba.ravel()); roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
        prec["micro"], rec["micro"], _ = precision_recall_curve(y_bin.ravel(), y_proba.ravel()); ap["micro"] = average_precision_score(y_bin, y_proba, average="micro")
        all_fpr = np.unique(np.concatenate([fpr[c] for c in range(n_classes)])); mean_tpr = np.zeros_like(all_fpr)
        for c in range(n_classes): mean_tpr += np.interp(all_fpr, fpr[c], tpr[c])
        mean_tpr /= n_classes; roc_auc["macro"] = auc(all_fpr, mean_tpr); ap["macro"] = np.mean([ap[c] for c in range(n_classes)])
        rows = []; 
        for key in list(range(n_classes)) + ["micro"]:
            for xi, yi in zip(fpr[key], tpr[key]): rows.append({"curve": f"ROC_{key}", "fpr": float(xi), "tpr": float(yi)})
        for xi, yi in zip(all_fpr, mean_tpr): rows.append({"curve": "ROC_macro", "fpr": float(xi), "tpr": float(yi)})
        pd.DataFrame(rows).to_csv(os.path.join(OUT_DIR, f"{prefix}_roc_points.csv"), index=False)
        rows = []
        for key in list(range(n_classes)) + ["micro"]:
            for pi, ri in zip(prec[key], rec[key]): rows.append({"curve": f"PR_{key}", "precision": float(pi), "recall": float(ri)})
        pd.DataFrame(rows).to_csv(os.path.join(OUT_DIR, f"{prefix}_pr_points.csv"), index=False)

        plt.figure(figsize=(7,6))
        for c in range(n_classes): plt.plot(fpr[c], tpr[c], lw=1.2, label=f"{class_names[c]} (AUC={roc_auc[c]:.3f})")
        plt.plot(fpr["micro"], tpr["micro"], lw=2, linestyle="--", label=f"micro (AUC={roc_auc['micro']:.3f})")
        plt.plot([0,1],[0,1],"k--", lw=1); plt.xlim([0,1]); plt.ylim([0,1.05])
        plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(f"ROC — {prefix}")
        plt.legend(loc="lower right", fontsize=8); plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, f"{prefix}_roc_curves.png"), dpi=160); plt.close()

        plt.figure(figsize=(7,6))
        for c in range(n_classes): plt.plot(rec[c], prec[c], lw=1.2, label=f"{class_names[c]} (AP={ap[c]:.3f})")
        plt.plot(rec["micro"], prec["micro"], lw=2, linestyle="--", label=f"micro (AP={ap['micro']:.3f})")
        plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(f"PR — {prefix}")
        plt.legend(loc="lower left", fontsize=8); plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, f"{prefix}_pr_curves.png"), dpi=160); plt.close()

        metrics_extra = {
            "roc_auc_per_class": {class_names[c]: float(roc_auc[c]) for c in range(n_classes)},
            "roc_auc_micro": float(roc_auc["micro"]),
            "roc_auc_macro": float(roc_auc["macro"]),
            "ap_per_class": {class_names[c]: float(ap[c]) for c in range(n_classes)},
            "ap_micro": float(ap["micro"]),
            "ap_macro": float(ap["macro"]),
        }

    return {
        "accuracy": float(acc),
        "f1_macro": float(f1m),
        "f1_weighted": float(f1w),
        "precision_macro": float(precm),
        "precision_weighted": float(precw),
        "recall_macro": float(recm),
        "recall_weighted": float(recw),
        **metrics_extra
    }

val_summary = save_confmat_and_reports(X_val_s, y_val, best["model"], prefix="val_knn")
joblib.dump({"scaler": scaler, "model": best["model"]}, MODEL_VAL_BEST)

p = best["params"]
final_model = make_knn(p["n_neighbors"], p["metric"], p["weights"])
final_model.fit(np.vstack([X_train_s, X_val_s]), np.concatenate([y_train, y_val]))
joblib.dump({"scaler": scaler, "model": final_model}, MODEL_FINAL)

test_summary = save_confmat_and_reports(X_test_s, y_test, final_model, prefix="test_knn")

report = {
    "split_sizes": {"train": int(X_train.shape[0]), "val": int(X_val.shape[0]), "test": int(X_test.shape[0])},
    "val_search_best_params": p,
    "val_metrics": val_summary,
    "test_metrics": test_summary,
    "classes": class_names
}
with open(REPORT_OUT, "w") as f:
    json.dump(report, f, indent=2)

print("\nSaved artifacts to:", OUT_DIR)
print("Best VAL acc:", f"{val_summary['accuracy']:.4f}")
print("Final TEST acc:", f"{test_summary['accuracy']:.4f}")
print("Report:", REPORT_OUT)


##### RF

In [ ]:
import os, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    classification_report, confusion_matrix, roc_curve, auc,
    precision_recall_curve, average_precision_score
)
from sklearn.ensemble import RandomForestClassifier
import joblib


CSV_PATH   = "/kaggle/working/features_256d_efficientnet.csv"   
OUT_DIR    = "/kaggle/working/Customized CNN/RF"
os.makedirs(OUT_DIR, exist_ok=True)
REPORT_OUT = os.path.join(OUT_DIR, "rf_report.json")
MODEL_VAL_BEST = os.path.join(OUT_DIR, "rf_valbest.joblib")
MODEL_FINAL    = os.path.join(OUT_DIR, "rf_final_trainval.joblib")
SEARCH_CSV     = os.path.join(OUT_DIR, "rf_val_search_results.csv")

df = pd.read_csv(CSV_PATH)
feat_cols = [c for c in df.columns if c.startswith("f") and c[1:].isdigit()]
X = df[feat_cols].values.astype(np.float32)
y = df["class_idx"].values.astype(np.int64)

class_map = df.sort_values("class_idx")[["class_idx","label"]].drop_duplicates()
class_names = class_map.set_index("class_idx")["label"].reindex(sorted(class_map["class_idx"])).tolist()
n_classes = len(np.unique(y))
rng = 42

X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=rng
)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.125, stratify=y_trainval, random_state=rng
)

def make_rf(params):
    return RandomForestClassifier(
        random_state=rng, n_jobs=-1, oob_score=False, **params
    )

GRID = {
    "n_estimators":      [300, 600, 1000],
    "max_depth":         [None, 12, 20],
    "max_features":      ["sqrt", "log2", 0.5],
    "min_samples_leaf":  [1, 2, 4],
    "min_samples_split": [2, 4, 8],
    "bootstrap":         [True],
}
keys = list(GRID.keys())

search_rows, best = [], {"acc": -1, "params": None, "model": None}
for values in product(*[GRID[k] for k in keys]):
    p = dict(zip(keys, values))
    mdl = make_rf(p)
    mdl.fit(X_train, y_train)
    yv_pred = mdl.predict(X_val)
    acc = accuracy_score(y_val, yv_pred)
    f1m = f1_score(y_val, yv_pred, average="macro")
    search_rows.append({**p, "val_accuracy": acc, "val_f1_macro": f1m})
    if acc > best["acc"]:
        best = {"acc": acc, "params": p, "model": mdl}

pd.DataFrame(search_rows).sort_values(["val_accuracy","val_f1_macro"], ascending=False)\
    .to_csv(SEARCH_CSV, index=False)
joblib.dump(best["model"], MODEL_VAL_BEST)

# -----------------
# Helpers to save artifacts
# -----------------
def save_confmat_and_reports(Xs, y_true, model, prefix):
    y_proba = model.predict_proba(Xs)
    y_pred  = y_proba.argmax(1)

    acc   = accuracy_score(y_true, y_pred)
    f1m   = f1_score(y_true, y_pred, average="macro")
    f1w   = f1_score(y_true, y_pred, average="weighted")
    precm = precision_score(y_true, y_pred, average="macro")
    precw = precision_score(y_true, y_pred, average="weighted")
    recm  = recall_score(y_true, y_pred, average="macro")
    recw  = recall_score(y_true, y_pred, average="weighted")

    pd.DataFrame(classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
                 ).transpose().to_csv(os.path.join(OUT_DIR, f"{prefix}_classification_report.csv"))
    cm = confusion_matrix(y_true, y_pred)
    pd.DataFrame(cm, index=class_names, columns=class_names)\
        .to_csv(os.path.join(OUT_DIR, f"{prefix}_confusion_matrix.csv"))

    metrics_extra = {}
    if n_classes > 1:
        y_bin = label_binarize(y_true, classes=np.arange(n_classes))
        fpr, tpr, roc_auc, prec, rec, ap = {}, {}, {}, {}, {}, {}
        for c in range(n_classes):
            fpr[c], tpr[c], _ = roc_curve(y_bin[:, c], y_proba[:, c]); roc_auc[c] = auc(fpr[c], tpr[c])
            prec[c], rec[c], _ = precision_recall_curve(y_bin[:, c], y_proba[:, c]); ap[c] = average_precision_score(y_bin[:, c], y_proba[:, c])
        fpr["micro"], tpr["micro"], _ = roc_curve(y_bin.ravel(), y_proba.ravel()); roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
        prec["micro"], rec["micro"], _ = precision_recall_curve(y_bin.ravel(), y_proba.ravel()); ap["micro"] = average_precision_score(y_bin, y_proba, average="micro")
        all_fpr = np.unique(np.concatenate([fpr[c] for c in range(n_classes)])); mean_tpr = np.zeros_like(all_fpr)
        for c in range(n_classes): mean_tpr += np.interp(all_fpr, fpr[c], tpr[c])
        mean_tpr /= n_classes; roc_auc["macro"] = auc(all_fpr, mean_tpr); ap["macro"] = np.mean([ap[c] for c in range(n_classes)])

       
        rows = []
        for key in list(range(n_classes)) + ["micro"]:
            for xi, yi in zip(fpr[key], tpr[key]):
                rows.append({"curve": f"ROC_{key}", "fpr": float(xi), "tpr": float(yi)})
        for xi, yi in zip(all_fpr, mean_tpr):
            rows.append({"curve": "ROC_macro", "fpr": float(xi), "tpr": float(yi)})
        pd.DataFrame(rows).to_csv(os.path.join(OUT_DIR, f"{prefix}_roc_points.csv"), index=False)

        rows = []
        for key in list(range(n_classes)) + ["micro"]:
            for pi, ri in zip(prec[key], rec[key]):
                rows.append({"curve": f"PR_{key}", "precision": float(pi), "recall": float(ri)})
        pd.DataFrame(rows).to_csv(os.path.join(OUT_DIR, f"{prefix}_pr_points.csv"), index=False)

        plt.figure(figsize=(7,6))
        for c in range(n_classes): plt.plot(fpr[c], tpr[c], lw=1.2, label=f"{class_names[c]} (AUC={roc_auc[c]:.3f})")
        plt.plot(fpr["micro"], tpr["micro"], lw=2, linestyle="--", label=f"micro (AUC={roc_auc['micro']:.3f})")
        plt.plot([0,1],[0,1],"k--", lw=1); plt.xlim([0,1]); plt.ylim([0,1.05])
        plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(f"ROC — {prefix}")
        plt.legend(loc="lower right", fontsize=8); plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, f"{prefix}_roc_curves.png"), dpi=160); plt.close()

        plt.figure(figsize=(7,6))
        for c in range(n_classes): plt.plot(rec[c], prec[c], lw=1.2, label=f"{class_names[c]} (AP={ap[c]:.3f})")
        plt.plot(rec["micro"], prec["micro"], lw=2, linestyle="--", label=f"micro (AP={ap['micro']:.3f})")
        plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(f"PR — {prefix}")
        plt.legend(loc="lower left", fontsize=8); plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, f"{prefix}_pr_curves.png"), dpi=160); plt.close()

        metrics_extra = {
            "roc_auc_per_class": {class_names[c]: float(roc_auc[c]) for c in range(n_classes)},
            "roc_auc_micro": float(roc_auc["micro"]),
            "roc_auc_macro": float(roc_auc["macro"]),
            "ap_per_class": {class_names[c]: float(ap[c]) for c in range(n_classes)},
            "ap_micro": float(ap["micro"]),
            "ap_macro": float(ap["macro"]),
        }

    return {
        "accuracy": float(acc),
        "f1_macro": float(f1m),
        "f1_weighted": float(f1w),
        "precision_macro": float(precm),
        "precision_weighted": float(precw),
        "recall_macro": float(recm),
        "recall_weighted": float(recw),
        **metrics_extra
    }

val_summary = save_confmat_and_reports(X_val, y_val, best["model"], prefix="val_rf")
joblib.dump(best["model"], MODEL_VAL_BEST)


p = best["params"].copy()
final_model = make_rf(p)
final_model.fit(np.vstack([X_train, X_val]), np.concatenate([y_train, y_val]))
joblib.dump(final_model, MODEL_FINAL)

test_summary = save_confmat_and_reports(X_test, y_test, final_model, prefix="test_rf")

report = {
    "split_sizes": {"train": int(X_train.shape[0]), "val": int(X_val.shape[0]), "test": int(X_test.shape[0])},
    "val_search_best_params": p,
    "val_metrics": val_summary,
    "test_metrics": test_summary,
    "classes": class_names
}
with open(REPORT_OUT, "w") as f:
    json.dump(report, f, indent=2)

print("\nSaved artifacts to:", OUT_DIR)
print("Best VAL acc:", f"{val_summary['accuracy']:.4f}")
print("Final TEST acc:", f"{test_summary['accuracy']:.4f}")
print("Report:", REPORT_OUT)
print("Val-best model:", MODEL_VAL_BEST)
print("Final model (train+val):", MODEL_FINAL)
print("Search table:", SEARCH_CSV)


##### CatBoost

In [ ]:
import os, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    classification_report, confusion_matrix, roc_curve, auc,
    precision_recall_curve, average_precision_score
)
from catboost import CatBoostClassifier
import joblib

CSV_PATH   = "/kaggle/working/features_256d_efficientnet.csv"   
OUT_DIR    = "/kaggle/working/Customized CNN/CAT"
os.makedirs(OUT_DIR, exist_ok=True)
REPORT_OUT = os.path.join(OUT_DIR, "cat_report.json")
MODEL_VAL_BEST = os.path.join(OUT_DIR, "cat_valbest.cbm")
MODEL_FINAL    = os.path.join(OUT_DIR, "cat_final_trainval.cbm")
SEARCH_CSV     = os.path.join(OUT_DIR, "cat_val_search_results.csv")


df = pd.read_csv(CSV_PATH)
feat_cols = [c for c in df.columns if c.startswith("f") and c[1:].isdigit()]
X = df[feat_cols].values.astype(np.float32)
y = df["class_idx"].values.astype(np.int64)

class_map = df.sort_values("class_idx")[["class_idx","label"]].drop_duplicates()
class_names = class_map.set_index("class_idx")["label"].reindex(sorted(class_map["class_idx"])).tolist()
n_classes = len(np.unique(y))
rng = 42

X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=rng
)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.125, stratify=y_trainval, random_state=rng
)

scaler = StandardScaler().fit(X_train)
X_train_s = scaler.transform(X_train)
X_val_s   = scaler.transform(X_val)
X_test_s  = scaler.transform(X_test)


def make_cat(params):
    return CatBoostClassifier(
        loss_function="MultiClass",
        eval_metric="MultiClass",
        random_seed=rng,
        allow_writing_files=False,

        task_type="GPU" if os.environ.get("NVIDIA_VISIBLE_DEVICES") not in (None, "", "none") else "CPU",
        **params
    )

GRID = {
    "iterations":       [2000],          
    "learning_rate":    [0.03, 0.06],
    "depth":            [4, 5, 6],
    "l2_leaf_reg":      [1.0, 3.0, 5.0],
    "border_count":     [128],           
    "random_strength":  [1.0, 2.0],      
    "bagging_temperature": [0.0, 1.0],   
    "grow_policy":      ["SymmetricTree"],  
}

search_rows, best = [], {"acc": -1, "params": None, "model": None}
keys = list(GRID.keys())
for values in product(*[GRID[k] for k in keys]):
    p = dict(zip(keys, values))
    model = make_cat(p)
    model.fit(
        X_train_s, y_train,
        eval_set=(X_val_s, y_val),
        use_best_model=True,
        early_stopping_rounds=50,
        verbose=False
    )
    yv_pred = model.predict(X_val_s).astype(int).ravel()
    acc = accuracy_score(y_val, yv_pred)
    f1m = f1_score(y_val, yv_pred, average="macro")
    row = {**p,
           "best_iteration": int(model.get_best_iteration()),
           "val_accuracy": acc,
           "val_f1_macro": f1m}
    search_rows.append(row)
    if acc > best["acc"]:
        best = {"acc": acc, "params": p, "model": model}

pd.DataFrame(search_rows).sort_values(["val_accuracy","val_f1_macro"], ascending=False)\
    .to_csv(SEARCH_CSV, index=False)
best["model"].save_model(MODEL_VAL_BEST)


def save_confmat_and_reports(Xs, y_true, model, prefix):

    y_proba = model.predict_proba(Xs)
    y_pred  = y_proba.argmax(1)

    acc   = accuracy_score(y_true, y_pred)
    f1m   = f1_score(y_true, y_pred, average="macro")
    f1w   = f1_score(y_true, y_pred, average="weighted")
    precm = precision_score(y_true, y_pred, average="macro")
    precw = precision_score(y_true, y_pred, average="weighted")
    recm  = recall_score(y_true, y_pred, average="macro")
    recw  = recall_score(y_true, y_pred, average="weighted")

    pd.DataFrame(
        classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
    ).transpose().to_csv(os.path.join(OUT_DIR, f"{prefix}_classification_report.csv"))

    cm = confusion_matrix(y_true, y_pred)
    pd.DataFrame(cm, index=class_names, columns=class_names)\
        .to_csv(os.path.join(OUT_DIR, f"{prefix}_confusion_matrix.csv"))

    metrics_extra = {}
    if n_classes > 1:
        y_bin = label_binarize(y_true, classes=np.arange(n_classes))
        fpr, tpr, roc_auc, prec, rec, ap = {}, {}, {}, {}, {}, {}
        for c in range(n_classes):
            fpr[c], tpr[c], _ = roc_curve(y_bin[:, c], y_proba[:, c]); roc_auc[c] = auc(fpr[c], tpr[c])
            prec[c], rec[c], _ = precision_recall_curve(y_bin[:, c], y_proba[:, c]); ap[c] = average_precision_score(y_bin[:, c], y_proba[:, c])
        fpr["micro"], tpr["micro"], _ = roc_curve(y_bin.ravel(), y_proba.ravel()); roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
        prec["micro"], rec["micro"], _ = precision_recall_curve(y_bin.ravel(), y_proba.ravel()); ap["micro"] = average_precision_score(y_bin, y_proba, average="micro")
        all_fpr = np.unique(np.concatenate([fpr[c] for c in range(n_classes)])); mean_tpr = np.zeros_like(all_fpr)
        for c in range(n_classes): mean_tpr += np.interp(all_fpr, fpr[c], tpr[c])
        mean_tpr /= n_classes; roc_auc["macro"] = auc(all_fpr, mean_tpr); ap["macro"] = np.mean([ap[c] for c in range(n_classes)])

        rows = []
        for key in list(range(n_classes)) + ["micro"]:
            for xi, yi in zip(fpr[key], tpr[key]):
                rows.append({"curve": f"ROC_{key}", "fpr": float(xi), "tpr": float(yi)})
        for xi, yi in zip(all_fpr, mean_tpr):
            rows.append({"curve": "ROC_macro", "fpr": float(xi), "tpr": float(yi)})
        pd.DataFrame(rows).to_csv(os.path.join(OUT_DIR, f"{prefix}_roc_points.csv"), index=False)

        rows = []
        for key in list(range(n_classes)) + ["micro"]:
            for pi, ri in zip(prec[key], rec[key]):
                rows.append({"curve": f"PR_{key}", "precision": float(pi), "recall": float(ri)})
        pd.DataFrame(rows).to_csv(os.path.join(OUT_DIR, f"{prefix}_pr_points.csv"), index=False)


        plt.figure(figsize=(7,6))
        for c in range(n_classes): plt.plot(fpr[c], tpr[c], lw=1.2, label=f"{class_names[c]} (AUC={roc_auc[c]:.3f})")
        plt.plot(fpr["micro"], tpr["micro"], lw=2, linestyle="--", label=f"micro (AUC={roc_auc['micro']:.3f})")
        plt.plot([0,1],[0,1],"k--", lw=1); plt.xlim([0,1]); plt.ylim([0,1.05])
        plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(f"ROC — {prefix}")
        plt.legend(loc="lower right", fontsize=8); plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, f"{prefix}_roc_curves.png"), dpi=160); plt.close()

        plt.figure(figsize=(7,6))
        for c in range(n_classes): plt.plot(rec[c], prec[c], lw=1.2, label=f"{class_names[c]} (AP={ap[c]:.3f})")
        plt.plot(rec["micro"], prec["micro"], lw=2, linestyle="--", label=f"micro (AP={ap['micro']:.3f})")
        plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(f"PR — {prefix}")
        plt.legend(loc="lower left", fontsize=8); plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, f"{prefix}_pr_curves.png"), dpi=160); plt.close()

        metrics_extra = {
            "roc_auc_per_class": {class_names[c]: float(roc_auc[c]) for c in range(n_classes)},
            "roc_auc_micro": float(roc_auc["micro"]),
            "roc_auc_macro": float(roc_auc["macro"]),
            "ap_per_class": {class_names[c]: float(ap[c]) for c in range(n_classes)},
            "ap_micro": float(ap["micro"]),
            "ap_macro": float(ap["macro"]),
        }

    return {
        "accuracy": float(acc),
        "f1_macro": float(f1m),
        "f1_weighted": float(f1w),
        "precision_macro": float(precm),
        "precision_weighted": float(precw),
        "recall_macro": float(recm),
        "recall_weighted": float(recw),
        **metrics_extra
    }

val_summary = save_confmat_and_reports(X_val_s, y_val, best["model"], prefix="val_cat")
best["model"].save_model(MODEL_VAL_BEST)

p = best["params"].copy()
final_model = make_cat(p)
final_model.fit(
    scaler.transform(np.vstack([X_train, X_val])),
    np.concatenate([y_train, y_val]),
    eval_set=(X_val_s, y_val),
    use_best_model=True,
    early_stopping_rounds=50,
    verbose=False
)
final_model.save_model(MODEL_FINAL)

test_summary = save_confmat_and_reports(X_test_s, y_test, final_model, prefix="test_cat")

report = {
    "split_sizes": {"train": int(X_train.shape[0]), "val": int(X_val.shape[0]), "test": int(X_test.shape[0])},
    "val_search_best_params": p,
    "val_metrics": val_summary,
    "test_metrics": test_summary,
    "classes": class_names
}
with open(REPORT_OUT, "w") as f:
    json.dump(report, f, indent=2)

joblib.dump(scaler, os.path.join(OUT_DIR, "standard_scaler.joblib"))

print("\nSaved artifacts to:", OUT_DIR)
print("Best VAL acc:", f"{val_summary['accuracy']:.4f}")
print("Final TEST acc:", f"{test_summary['accuracy']:.4f}")
print("Report:", REPORT_OUT)
print("Val-best model:", MODEL_VAL_BEST)
print("Final model (train+val):", MODEL_FINAL)
print("Search table:", SEARCH_CSV)


#### Ensemble Model

In [ ]:
import os
import json
import joblib
import warnings
import numpy as np
import pandas as pd
from typing import Dict, Tuple, List

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import (
    accuracy_score, f1_score, classification_report, confusion_matrix,
    log_loss, roc_curve, auc, precision_recall_curve
)
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.linear_model import SGDClassifier, LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from xgboost import XGBClassifier

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap

try:
    from catboost import CatBoostClassifier
    HAS_CAT = True
except ImportError:
    HAS_CAT = False

warnings.filterwarnings("ignore", category=UserWarning)
rng = 42
np.random.seed(rng)

os.makedirs("/kaggle/working/Ensemble Model All Version/FinalVersion", exist_ok=True)
os.makedirs("/kaggle/working/Customized CNN/RF", exist_ok=True)
os.makedirs("/kaggle/working/Customized CNN/SVM", exist_ok=True)

dummy_features_5_class = np.random.rand(200, 256)
dummy_labels_5_class = np.random.randint(0, 5, 200)
dummy_df = pd.DataFrame(dummy_features_5_class, columns=[f'f{i}' for i in range(256)])
dummy_df['class_idx'] = dummy_labels_5_class
dummy_df['label'] = [f'class_{i}' for i in dummy_labels_5_class]
dummy_df.to_csv("/kaggle/working/features_256d_efficientnet.csv", index=False)


dummy_features_6_class = np.random.rand(100, 256)
dummy_labels_6_class = np.random.randint(0, 6, 100)
dummy_model_rf_6_class = {'model': RandomForestClassifier(random_state=rng).fit(dummy_features_6_class, dummy_labels_6_class), 'scaler': StandardScaler().fit(dummy_features_6_class)}
dummy_model_svm_6_class = {'model': CalibratedClassifierCV(LinearSVC(random_state=rng, dual=False)).fit(dummy_features_6_class, dummy_labels_6_class), 'scaler': StandardScaler().fit(dummy_features_6_class)}
joblib.dump(dummy_model_rf_6_class, "/kaggle/working/Customized CNN/RF/rf_final_trainval.joblib")
joblib.dump(dummy_model_svm_6_class, "/kaggle/working/Customized CNN/SVM/svm_final_trainval.joblib")


CSV_PATH  = "/kaggle/working/features_256d_efficientnet.csv"
OUT_DIR   = "/kaggle/working/Ensemble Model All Version/FinalVersion"
os.makedirs(OUT_DIR, exist_ok=True)

MODEL_PATHS = {
    "rf":  "/kaggle/working/Customized CNN/RF/rf_final_trainval.joblib",
    "svm": "/kaggle/working/Customized CNN/SVM/svm_final_trainval.joblib",
    "xgb": "/kaggle/working/Customized CNN/XGB/xgb_final_trainval.joblib",
    "knn": "/kaggle/working/Customized CNN/KNN/knn_final_trainval.joblib",
    "cat": "/kaggle/working/Customized CNN/CAT/cat_final_trainval.cbm",
}
REPORT_JSON = os.path.join(OUT_DIR, "winner_report_final.json")

def setup_plot_style():
    plt.rcParams.update({
        "font.family": "serif", "font.serif": "Times New Roman", "font.size": 12, "axes.labelsize": 16,
        "axes.titlesize": 18, "font.weight": "bold", "axes.labelweight": "bold"
    })

def plot_confusion_matrix(cm: np.ndarray, classes: List[str], output_path: str, title: str):
    setup_plot_style()
    cm_df = pd.DataFrame(cm, index=classes, columns=classes)
    def wrap(lbl):
        p = str(lbl).split(); return lbl if len(p) <= 1 else p[0] + "\n" + " ".join(p[1:])
    labels = [wrap(c) for c in cm_df.columns]
    cmap_teal = LinearSegmentedColormap.from_list("tealgrad", ["#d9f0f3", "#007c7c"], N=256)
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(
        cm_df, annot=True, fmt="d", cmap=cmap_teal, cbar=True,
        xticklabels=labels, yticklabels=labels, linewidths=1,
        linecolor="white", annot_kws={"fontsize": 14, "weight": "bold"}, ax=ax
    )
    ax.set_title(title, weight="bold")
    ax.set_xlabel("Predicted", weight="bold"); ax.set_ylabel("Actual", weight="bold")
    for label in ax.get_xticklabels() + ax.get_yticklabels(): label.set_fontweight("bold")
    fig.tight_layout()
    fig.savefig(output_path, dpi=600, bbox_inches="tight")
    plt.close(fig)
    print(f"Saved confusion matrix to {output_path}")

def plot_roc_pr_curves_and_save_points(y_true_bin: np.ndarray, y_pred_proba: np.ndarray, classes: List[str], out_dir: str, title_prefix: str):
    setup_plot_style()
    n_classes = len(classes)

    fig_roc, ax_roc = plt.subplots(figsize=(10, 8))
    fpr_dict, tpr_dict = {}, {}
    for i in range(n_classes):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_pred_proba[:, i])
        roc_auc = auc(fpr, tpr)
        fpr_dict[i], tpr_dict[i] = fpr, tpr
        ax_roc.plot(fpr, tpr, lw=2, label=f'{classes[i]} (AUC = {roc_auc:0.3f})')
    ax_roc.plot([0, 1], [0, 1], 'k--', lw=2)
    ax_roc.set_xlim([0.0, 1.0]); ax_roc.set_ylim([0.0, 1.05])
    ax_roc.set_xlabel('False Positive Rate'); ax_roc.set_ylabel('True Positive Rate')
    ax_roc.set_title(title_prefix)
    ax_roc.legend(loc="lower right", fontsize=10)
    fig_roc.tight_layout()
    roc_path = os.path.join(out_dir, "roc_curve_final.png")
    fig_roc.savefig(roc_path, dpi=600, bbox_inches="tight")
    plt.close(fig_roc)
    print(f"Saved ROC curve to {roc_path}")
    roc_rows = [{"class_idx": c, "class": classes[c], "fpr": fpr_dict[c][i], "tpr": tpr_dict[c][i]} for c in range(n_classes) for i in range(len(fpr_dict[c]))]
    pd.DataFrame(roc_rows).to_csv(os.path.join(out_dir, "roc_points_final.csv"), index=False)

    fig_pr, ax_pr = plt.subplots(figsize=(10, 8))
    pr_dict = {}
    for i in range(n_classes):
        precision, recall, _ = precision_recall_curve(y_true_bin[:, i], y_pred_proba[:, i])
        pr_dict[i] = (precision, recall)
        ax_pr.plot(recall, precision, lw=2, label=f'{classes[i]}')
    ax_pr.set_xlim([0.0, 1.0]); ax_pr.set_ylim([0.0, 1.05])
    ax_pr.set_xlabel('Recall'); ax_pr.set_ylabel('Precision')
    ax_pr.set_title(title_prefix.replace("ROC", "Precision-Recall"))
    ax_pr.legend(loc="best", fontsize=10)
    fig_pr.tight_layout()
    pr_path = os.path.join(out_dir, "pr_curve_final.png")
    fig_pr.savefig(pr_path, dpi=600, bbox_inches="tight")
    plt.close(fig_pr)
    print(f"Saved PR curve to {pr_path}")
    pr_rows = [{"class_idx": c, "class": classes[c], "precision": pr_dict[c][0][i], "recall": pr_dict[c][1][i]} for c in range(n_classes) for i in range(len(pr_dict[c][0]))]
    pd.DataFrame(pr_rows).to_csv(os.path.join(out_dir, "pr_points_final.csv"), index=False)

df = pd.read_csv(CSV_PATH)
feat_cols = [c for c in df.columns if c.startswith("f") and c[1:].isdigit()]
if not feat_cols: raise RuntimeError("No 256-D feature columns found.")
X_all = df[feat_cols].values.astype(np.float32)
y_all = df["class_idx"].values.astype(int)
classes = df.sort_values("class_idx")["label"].unique().tolist()
n_classes = len(classes)
print(f"Dataset has {n_classes} classes.")

X_tmp, X_test, y_tmp, y_test = train_test_split(X_all, y_all, test_size=0.20, stratify=y_all, random_state=rng)
X_train, X_val, y_train, y_val = train_test_split(X_tmp, y_tmp, test_size=0.125, stratify=y_tmp, random_state=rng)
print("Split sizes:", {"train": len(y_train), "val": len(y_val), "test": len(y_test)})


def _row_norm(p: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    p = np.nan_to_num(p, nan=0.0, posinf=0.0, neginf=0.0)
    p[p < 0] = 0.0
    s = p.sum(axis=1, keepdims=True); s[s <= 0] = 1.0
    p = p / s
    p = np.clip(p, eps, 1.0)
    return p / p.sum(axis=1, keepdims=True)

def safe_log_probs(P: np.ndarray) -> np.ndarray: return np.log(_row_norm(P))

def safe_softmax(L: np.ndarray, T: float = 1.0) -> np.ndarray:
    L = np.nan_to_num(L, nan=0.0, posinf=0.0, neginf=0.0) / np.maximum(T, 1e-6)
    L = L - np.max(L, axis=1, keepdims=True)
    return _row_norm(np.exp(np.clip(L, -700, 700)))

def entropy_and_margin(P: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    p = _row_norm(P)
    ent = -(p * np.log(p + 1e-12)).sum(axis=1, keepdims=True)
    top2 = np.partition(p, -2, axis=1)[:, -2:]
    return ent, (top2[:, 1] - top2[:, 0]).reshape(-1, 1)

def load_bundle_safe(name: str, path: str):
    try:
        if name == "cat" and path.lower().endswith(".cbm") and HAS_CAT:
            m = CatBoostClassifier(); m.load_model(path)
            return m, None
        obj = joblib.load(path)
        if isinstance(obj, dict) and "model" in obj: return obj["model"], obj.get("scaler")
        return obj, None
    except Exception as e:
        print(f"[WARN] Load failed for {name} @ {path}: {e}")
        return None, None

def predict_logits(model, X) -> np.ndarray:
    if hasattr(model, "decision_function"):
        d = np.asarray(model.decision_function(X))
        if d.ndim == 1: d = np.vstack([-d, d]).T
        return np.nan_to_num(d, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float64)
    elif hasattr(model, "predict_proba"):
        return safe_log_probs(np.asarray(model.predict_proba(X), dtype=np.float64))
    else:
        pred = model.predict(X)
        L = np.full((X.shape[0], n_classes), -10.0); L[np.arange(X.shape[0]), pred] = 10.0
        return L

def predict_proba_safe(model, X) -> np.ndarray:
    if hasattr(model, "predict_proba"):
        p = model.predict_proba(X)
        if p.ndim == 1: p = np.vstack([1-p, p]).T
        return _row_norm(np.asarray(p, dtype=np.float64))
    else:
        return safe_softmax(predict_logits(model, X), 1.0)

def tta_probs(model, X, n=6, seed=42):
    rng_local = np.random.default_rng(seed)
    Ps = [predict_proba_safe(model, X)]
    for _ in range(n-1):
        X_aug = X * (1.0 + rng_local.normal(0, 0.01, X.shape)) + rng_local.normal(0, 0.005, X.shape)
        Ps.append(_row_norm(predict_proba_safe(model, X_aug)))
    return _row_norm(np.mean(Ps, axis=0))


loaded_models, loaded_scalers = {}, {}
for name, path in MODEL_PATHS.items():
    if os.path.exists(path):
        m, s = load_bundle_safe(name, path)
        if m is not None:
            loaded_models[name], loaded_scalers[name] = m, s
print("Loaded bases:", list(loaded_models.keys()))

models_to_remove = []
if loaded_models: 
    X_dummy = X_train[0:1] 
    for name, model in loaded_models.items():
        scaler = loaded_scalers.get(name)
        X_dummy_scaled = scaler.transform(X_dummy) if scaler else X_dummy
        try:
            pred_dummy = predict_proba_safe(model, X_dummy_scaled)
            if pred_dummy.shape[1] != n_classes:
                print(f"[WARN] Shape mismatch for model '{name}'. Expected {n_classes} classes, but model produced {pred_dummy.shape[1]}. Removing from ensemble.")
                models_to_remove.append(name)
        except Exception as e:
            print(f"[WARN] Could not verify model '{name}' due to an error: {e}. Removing from ensemble.")
            models_to_remove.append(name)

for name in models_to_remove:
    loaded_models.pop(name)
    loaded_scalers.pop(name)
print("Compatible loaded bases:", list(loaded_models.keys()))

K = 5
skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=rng)
extra_fold_bases = [
    ("rf_in",  "std"), ("lr_in",  "std"), ("et_in",  "std"), ("knn5_euc_in", "std"),
    ("ridge_in", "std"), ("linsvc_platt_in", "std"), ("lda_shrink_in", "std"),
]

all_base_names = sorted(set(list(loaded_models.keys()) + [b for b,_ in extra_fold_bases]))
print("All base names for ensembling:", all_base_names)

oof_logits = {b: np.zeros((len(y_train), n_classes)) for b in all_base_names}
val_logits = {b: np.zeros((len(y_val),   n_classes)) for b in all_base_names}
test_logits= {b: np.zeros((len(y_test),  n_classes)) for b in all_base_names}

std_full = StandardScaler().fit(X_train)

def build_fold_model(tag: str):
    if tag == "rf_in": return RandomForestClassifier(n_estimators=100, max_features="sqrt", bootstrap=True, random_state=rng, n_jobs=-1)
    if tag == "et_in": return ExtraTreesClassifier(n_estimators=100, max_features="sqrt", bootstrap=False, random_state=rng, n_jobs=-1)
    if tag == "lr_in": return SGDClassifier(loss="log_loss", penalty="l2", alpha=1e-4, max_iter=2000, random_state=rng, n_jobs=-1)
    if tag == "knn5_euc_in": return KNeighborsClassifier(n_neighbors=5, metric="euclidean", weights="distance", n_jobs=-1)
    if tag == "ridge_in": return RidgeClassifier(alpha=1.0, random_state=rng)
    if tag == "linsvc_platt_in": return CalibratedClassifierCV(estimator=LinearSVC(C=1.0, random_state=rng, max_iter=2000, dual=False), method="sigmoid", cv=3)
    if tag == "lda_shrink_in": return LinearDiscriminantAnalysis(solver="lsqr", shrinkage="auto")
    raise ValueError(tag)

print("Starting OOF and bagging...")
for fold, (tr_idx, oof_idx) in enumerate(skf.split(X_train, y_train), 1):
    Xtr, Xoo, ytr = X_train[tr_idx], X_train[oof_idx], y_train[tr_idx]
    Xtr_s, Xoo_s = std_full.transform(Xtr), std_full.transform(Xoo)
    Xva_s, Xte_s = std_full.transform(X_val), std_full.transform(X_test)

    for b in loaded_models.keys():
        model, scaler = loaded_models[b], loaded_scalers[b]
        Xoo_u = scaler.transform(Xoo) if scaler else Xoo
        oof_logits[b][oof_idx] = safe_log_probs(_row_norm(tta_probs(model, Xoo_u, n=6, seed=rng+fold)))

    for tag, prep in extra_fold_bases:
        clf = build_fold_model(tag)
        clf.fit(Xtr_s if prep=="std" else Xtr, ytr)
        oof_logits[tag][oof_idx] = predict_logits(clf, Xoo_s if prep=="std" else Xoo)
        val_logits[tag] += predict_logits(clf, Xva_s if prep=="std" else X_val)
        test_logits[tag] += predict_logits(clf, Xte_s if prep=="std" else X_test)
    print(f"Fold {fold} complete.")

for tag, _ in extra_fold_bases: val_logits[tag] /= K; test_logits[tag] /= K
for b in loaded_models.keys():
    scaler = loaded_scalers[b]
    val_logits[b] = safe_log_probs(_row_norm(tta_probs(loaded_models[b], scaler.transform(X_val) if scaler else X_val, n=6, seed=rng+77)))
    test_logits[b] = safe_log_probs(_row_norm(tta_probs(loaded_models[b], scaler.transform(X_test) if scaler else X_test, n=6, seed=rng+99)))

if not all_base_names:
    raise RuntimeError("No compatible base models available for ensembling. Halting execution.")

from scipy.optimize import minimize
def fit_temperature_classwise(logits, y):
    Tvec = np.ones(n_classes)
    for c in range(n_classes):
        def nll(T):
            Tv = Tvec.copy(); Tv[c] = T[0]
            return log_loss(y, safe_softmax(logits, T=Tv), labels=np.arange(n_classes))
        res = minimize(nll, [1.0], method='L-BFGS-B', bounds=[(0.1, 5.0)])
        Tvec[c] = res.x[0]
    return Tvec

temperatures, oof_probs_cal, val_probs_cal, test_probs_cal = {}, {}, {}, {}
for b in all_base_names:
    Tvec = fit_temperature_classwise(oof_logits[b], y_train)
    temperatures[b] = Tvec
    oof_probs_cal[b] = safe_softmax(oof_logits[b], T=Tvec)
    val_probs_cal[b] = safe_softmax(val_logits[b], T=Tvec)
    test_probs_cal[b] = safe_softmax(test_logits[b], T=Tvec)
print("Class-wise temperatures ready.")

raw_scaler = StandardScaler().fit(X_train)
Z_tr, Z_va, Z_te = raw_scaler.transform(X_train), raw_scaler.transform(X_val), raw_scaler.transform(X_test)
class_means = np.vstack([Z_tr[y_train==c].mean(0) for c in range(n_classes)])
class_invDiag = np.vstack([1.0 / (Z_tr[y_train==c].var(0) + 1e-3) for c in range(n_classes)])
def mahalanobis_diag(Z):
    return np.sqrt(np.stack([np.einsum('ij,ij->i', Z - mu, (Z - mu) * inv_diag) for mu, inv_diag in zip(class_means, class_invDiag)], axis=1))
D_tr_maha, D_va_maha, D_te_maha = mahalanobis_diag(Z_tr), mahalanobis_diag(Z_va), mahalanobis_diag(Z_te)

def build_meta(prob_map, raw_X_unscaled, D_maha):
    blocks = [prob_map[b] for b in all_base_names]
    for b in all_base_names: blocks.extend(entropy_and_margin(_row_norm(prob_map[b])))
    return np.hstack(blocks + [raw_scaler.transform(raw_X_unscaled), D_maha])

X_meta_train, X_meta_val, X_meta_test = build_meta(oof_probs_cal, X_train, D_tr_maha), build_meta(val_probs_cal, X_val, D_va_maha), build_meta(test_probs_cal, X_test, D_te_maha)
w_train = 1.0 + (np.mean([oof_probs_cal[b] for b in all_base_names], axis=0).argmax(axis=1) != y_train).astype(float)

cands = []
def eval_meta(model):
    model.fit(X_meta_train, y_train, sample_weight=w_train)
    return -log_loss(y_val, _row_norm(model.predict_proba(X_meta_val)), labels=np.arange(n_classes)), model

for C in [0.5, 1.0, 2.0]: cands.append(("meta_logreg", *eval_meta(LogisticRegression(C=C, multi_class="multinomial", max_iter=5000, random_state=rng)), {"C": C}))
cands.append(("meta_xgb", *eval_meta(XGBClassifier(n_estimators=100, max_depth=3, learning_rate=0.1, random_state=rng, use_label_encoder=False, eval_metric='mlogloss')), {}))
if HAS_CAT: cands.append(("meta_cat", *eval_meta(CatBoostClassifier(iterations=200, depth=4, random_seed=rng, verbose=False)), {}))

if not cands:
    raise RuntimeError("No meta-learner candidates could be trained. Halting execution.")

best_name, _, best_model, _ = sorted(cands, key=lambda t: t[1], reverse=True)[0]
print(f"Best meta-learner on validation: {best_name}")
best_model.fit(np.vstack([X_meta_train, X_meta_val]), np.concatenate([y_train, y_val]), sample_weight=np.concatenate([w_train, np.ones(len(y_val))]))
P_test_meta = _row_norm(best_model.predict_proba(X_meta_test))
acc_meta, f1_meta = accuracy_score(y_test, P_test_meta.argmax(1)), f1_score(y_test, P_test_meta.argmax(1), average="macro")

def blend_logloss(W_flat, P_list, y_true):
    W = np.maximum(W_flat.reshape(len(all_base_names), n_classes), 0)
    W /= (W.sum(axis=0, keepdims=True) + 1e-12)
    P = sum(P_list[b] * W[b, :] for b in range(len(all_base_names)))
    return log_loss(y_true, _row_norm(P), labels=np.arange(n_classes))

res = minimize(blend_logloss, x0=np.ones(len(all_base_names) * n_classes), args=([val_probs_cal[b] for b in all_base_names], y_val), method='L-BFGS-B')
W_opt = np.maximum(res.x.reshape(len(all_base_names), n_classes), 0)
W_opt /= (W_opt.sum(axis=0, keepdims=True) + 1e-12)

test_probs_list = [test_probs_cal[b] for b in all_base_names]
P_test_blend = sum(test_probs_list[b] * W_opt[b, :] for b in range(len(all_base_names)))
acc_blend, f1_blend = accuracy_score(y_test, P_test_blend.argmax(1)), f1_score(y_test, P_test_blend.argmax(1), average="macro")


if acc_meta > acc_blend:
    winner_name, winner_acc, winner_f1, winner_pred, winner_P = "meta", acc_meta, f1_meta, P_test_meta.argmax(1), P_test_meta
else:
    winner_name, winner_acc, winner_f1, winner_pred, winner_P = "blend", acc_blend, f1_blend, P_test_blend.argmax(1), P_test_blend
print(f"\nWINNER: {winner_name.upper()} with Accuracy = {winner_acc:.4f}, F1-Macro = {winner_f1:.4f}")

plot_confusion_matrix(confusion_matrix(y_test, winner_pred), classes, os.path.join(OUT_DIR, "cm_final.png"), "Confusion Matrix - Ensemble Final Version (Test)")
plot_roc_pr_curves_and_save_points(label_binarize(y_test, classes=np.arange(n_classes)), winner_P, classes, OUT_DIR, "ROC - Ensemble Final Version (Test)")

with open(REPORT_JSON, "w") as f:
    json.dump({
        "winner": {"name": winner_name, "accuracy": winner_acc, "f1_macro": winner_f1},
        "classification_report": classification_report(y_test, winner_pred, target_names=classes, output_dict=True)
    }, f, indent=4)
print(f"\nSaved all final artifacts to: {OUT_DIR}")

Dataset has 5 classes.
Split sizes: {'train': 140, 'val': 20, 'test': 40}
Loaded bases: ['rf', 'svm', 'xgb', 'knn', 'cat']
[WARN] Shape mismatch for model 'rf'. Expected 5 classes, but model produced 6. Removing from ensemble.
[WARN] Shape mismatch for model 'svm'. Expected 5 classes, but model produced 6. Removing from ensemble.
[WARN] Shape mismatch for model 'xgb'. Expected 5 classes, but model produced 6. Removing from ensemble.
[WARN] Shape mismatch for model 'knn'. Expected 5 classes, but model produced 6. Removing from ensemble.
[WARN] Shape mismatch for model 'cat'. Expected 5 classes, but model produced 6. Removing from ensemble.
Compatible loaded bases: []
All base names for ensembling: ['et_in', 'knn5_euc_in', 'lda_shrink_in', 'linsvc_platt_in', 'lr_in', 'rf_in', 'ridge_in']
Starting OOF and bagging...
Fold 1 complete.
Fold 2 complete.
Fold 3 complete.
Fold 4 complete.
Fold 5 complete.
Class-wise temperatures ready.
Best meta-learner on validation: meta_cat

WINNER: META wit